<a href="https://colab.research.google.com/github/Alirezamirbagheri/emotor-assembly-inspection-ml/blob/main/emotor-assembly-inspection-ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mounting the Gdrive
from google.colab import drive
from google.colab.patches import cv2_imshow
import sys, os
drive.mount('/content/gdrive')

# Changing and adjusting the working directory = should be customized
dir_path = '/content/gdrive/My Drive/ML/Image_classification/Emotor_inspection'
sys.path.append(dir_path)
os.chdir(dir_path)
